# K-NN  Classifier

## Imports

In [15]:
import pandas as pd
from sklearn import datasets as ds
from sklearn.model_selection import train_test_split as tts
import math

## Load data

In [16]:
iris = ds.load_iris()
df_x = pd.DataFrame(data= iris.data, columns= iris.feature_names)
df_y = iris.target


In [17]:
df_x

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [18]:
df_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Split data into training and test sets

In [19]:
x_train, x_test, y_train, _y_test  = tts(df_x, df_y, test_size= 0.2, random_state= 0)

## Implement Eculidean distance calculator

In [20]:
def euclidean_distance(new_obj, train_obj):
    distance = math.sqrt(sum([(new_obj[i] - train_obj[i])**2 for i in range(len(new_obj)-1)]))
    return distance

## Implement classification method

In [21]:
def classify(distance, K):
    selected_class = distance.iloc[0:K].pivot_table(columns= "class", aggfunc= "size")
    return selected_class.idxmax()

## Implement K-NN algorithm

In [22]:
def knn_algorithm(x_train, x_test, y_train, K):
    y_predictions = []
    for i in range(len(x_test)):
        euclidean_distances = [euclidean_distance(x_test.iloc[i], x_train.iloc[j]) for j in range(len(x_train))]
        distance = pd.DataFrame({"distance": euclidean_distances,
                                 "class": y_train})
        distance.sort_values(by= "distance", ascending= True, inplace= True)
        y_predictions.append(classify(distance, K))
    x_test['class'] = y_predictions
    return x_test

## Prediction attempt

In [23]:
prediction = knn_algorithm(x_train= x_train, x_test= x_test, y_train= y_train, K= 5)
    

In [24]:
prediction

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class
114,5.8,2.8,5.1,2.4,2
62,6.0,2.2,4.0,1.0,1
33,5.5,4.2,1.4,0.2,0
107,7.3,2.9,6.3,1.8,2
7,5.0,3.4,1.5,0.2,0
100,6.3,3.3,6.0,2.5,2
40,5.0,3.5,1.3,0.3,0
86,6.7,3.1,4.7,1.5,1
76,6.8,2.8,4.8,1.4,1
71,6.1,2.8,4.0,1.3,1
